In [2]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [13]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# test.csv 파일을 읽어옵니다.
df = pd.read_csv('test.csv')

In [17]:
import torch

device = 0 if torch.cuda.is_available() else -1
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device=device)

In [18]:
results = []

# 각 질문에 대해 응답 생성
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Generating Responses"):
    prompt = row['Question']
    
    # 텍스트 생성
    outputs = pipe_finetuned(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        add_special_tokens=True
    )
    
    # 생성된 텍스트에서 프롬프트를 제외한 부분만 추출
    generated_text = outputs[0]['generated_text']
    response = generated_text[len(prompt):].strip()  # 프롬프트 부분을 제외하고 응답만 추출

    # 결과를 출력하고 리스트에 저장
    print(f"Question: {prompt}\n")
    print(f"Answer: {response}\n")
    
    results.append({
        'SAMPLE_ID': row['SAMPLE_ID'],
        'Answer': response
    })

Generating Responses:   1%|          | 1/98 [00:09<16:04,  9.94s/it]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,022억원입니다.



Generating Responses:   2%|▏         | 2/98 [00:16<13:07,  8.20s/it]

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?

Answer: ---

중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 단계별로 필요한 자금을 지원하는 것이다.



Generating Responses:   3%|▎         | 3/98 [00:34<19:52, 12.55s/it]

...

Question: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?

Answer: ### 국가결산보고서

* 우발부채는 '예산 증감내역'에서 '기타국세수입 감소' 항목으로 분류하여 보고합니다.

### 지방자치단체 회계기준

* 우발부채는 '회계변동내역'에서 '기타국세수입 감소' 항목으로 분류하여 보고합니다.



Generating Responses:  99%|█████████▉| 97/98 [20:59<00:10, 10.96s/it]

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?

Answer: ### 정의: 우발부채는 기업의 일반적인 경영활동에서 비롯되지 않고, 예상치 못한 사유로 발생하는 부채를 의미합니다.

### 중요성: 우발부채는 기업의 재정안정성을 저해할 수 있는 주요 원인 중 하나이며, 적절한 관리를 통해 기업의 재정건전성을 유지하는 것이 중요합니다.



Generating Responses: 100%|██████████| 98/98 [21:12<00:00, 12.98s/it]

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?

Answer: ### Response: 보증은 채무자가 채무를 이행하지 못할 경우, 보증자가 대신해서 갚아야 하는 약속을 의미합니다. 재정상태표에는 채무자가 채무를 이행하지 못할 경우 보증자가 대신해서 갚아야 하는 의무가 된 후에, 보증금액만큼의 채무로 기록됩니다. 표준화 보증은 국민권익위원회의 의결을 얻지 않으면 발행할 수 없는 보증이며, 중소기업 및 스타트업의 경영난 해소, 자금 조달 시설의 개선 등의 목적으로 발행됩니다.



In [20]:
df_results = pd.DataFrame(results)
df_results.to_csv('gemma2_instruction_tuning_v1.csv', index=False)